In [ ]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

import phik
from phik import report, resources
from phik.binning import bin_data
from phik.report import plot_correlation_matrix

In [ ]:
df = pd.read_excel('../../data/coorte-rues-2022-2023-V3_com_extracoes.xlsx')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
tradução = {
    'prontuario': 'medical_record_id',
    'n_internacao': 'admission_number',
    'data_internacao': 'admission_date',
    'data_saida': 'discharge_date',
    'tipo_alta': 'discharge_type',
    'tempo_permanencia': 'length_of_stay',
    'tempo_ultima_internacao_internacao_atual': 'time_since_last_admission',
    'internacoes_ultimos_365_dias': 'admissions_last_365_days',
    'internacoes_ultimos_30_dias': 'admissions_last_30_days',
    'internacoes_ultimos_7_dias': 'admissions_last_7_days',
    'internacoes_365_dias_depois_alta': 'readmissions_365_days_post_discharge',
    'internacoes_30_dias_depois_alta': 'readmissions_30_days_post_discharge',
    'internacoes_7_dias_depois_alta': 'readmissions_7_days_post_discharge',
    'total_diagnostico_principal_alta': 'primary_diagnosis_count_at_discharge',
    'total_diagnostico_secundario_alta': 'secondary_diagnosis_count_at_discharge',
    'lto_lto_id': 'bed_id',
    'unidade_internacao': 'admission_unit',
    'especialidade_internacao': 'admission_specialty',
    'data_nascimento': 'birth_date',
    'idade_internacao': 'age_at_admission',
    'sexo': 'sex',
    'raca_cor': 'race_color',
    'naturalidade': 'place_of_birth',
    'nacionalidade': 'nationality',
    'municipio_residencia': 'residence_city',
    'estado_civil': 'marital_status',
    'grau_instrucao': 'education_level',
    'transferencias_durante_a_internacao': 'transfers_during_admission',
    'unidades_que_passou': 'units_visited',
    'especialidades_que_passou': 'specialties_visited',
    'cid_primario_internacao': 'primary_diagnosis_at_admission',
    'cid_secundario_internacao': 'secondary_diagnosis_at_admission',
    'diagnostico_principal_alta': 'primary_diagnosis_at_discharge',
    'diagnostico_secundario_alta': 'secondary_diagnosis_at_discharge',
    'primeiro_albumina': 'first_albumin',
    'ultimo_albumina': 'last_albumin',
    'primeiro_c_reativa': 'first_c_reactive_protein',
    'ultimo_c_reativa': 'last_c_reactive_protein',
    'primeiro_sodio': 'first_sodium',
    'ultimo_sodio': 'last_sodium',
    'primeiro_ureia': 'first_urea',
    'ultimo_ureia': 'last_urea',
    'primeiro_hemoglobina': 'first_hemoglobin',
    'ultimo_hemoglobina': 'last_hemoglobin',
    'primeiro_creatinina': 'first_creatinine',
    'ultimo_creatinina': 'last_creatinine',
    'primeiro_leococitos_hemograma': 'first_white_blood_cell_count',
    'ultimo_leococitos_hemograma': 'last_white_blood_cell_count',
    
    # sinais vitais - anamnese médica
    'FC__anamnese_medica': 'heart_rate_medical',
    'FR__anamnese_medica': 'respiratory_rate_medical',
    'PA__anamnese_medica': 'blood_pressure_medical',
    'T__anamnese_medica': 'temperature_medical',
    'SpO2__anamnese_medica': 'oxygen_saturation_medical',
    'R__anamnese_medica': 'consciousness_level_medical',

    # sinais vitais - enfermagem
    'FC__anamnese_enfermagem': 'heart_rate_nursing',
    'FR__anamnese_enfermagem': 'respiratory_rate_nursing',
    'PA__anamnese_enfermagem': 'blood_pressure_nursing',
    'T__anamnese_enfermagem': 'temperature_nursing',
    'SpO2__anamnese_enfermagem': 'oxygen_saturation_nursing',
    'R__anamnese_enfermagem': 'consciousness_level_nursing'
}

df = df.rename(columns=tradução)

##### Mantendo apenas a última internação de cada paciente

In [ ]:
df = df.sort_values(by=['medical_record_id', 'admission_date']).drop_duplicates(subset='medical_record_id', keep='last') 

In [ ]:
df[['blood_pressure_nursing', 'oxygen_saturation_nursing']].nunique()

##### Colunas que não farão parte do modelo:

In [ ]:
colunas_retiradas = [   
    
     'medical_record_id',
    'admission_date',
    'admission_number',
    'discharge_date',
    'primary_diagnosis_count_at_discharge',
    'secondary_diagnosis_count_at_discharge',
    'bed_id',
    'admission_unit',
    'birth_date',
    'place_of_birth',
    'nationality',
    'residence_city',
    'marital_status',
    'transfers_during_admission',
    'units_visited',
    'specialties_visited',

    # coluna com valor único
    'oxygen_saturation_nursing',
    'blood_pressure_medical',

    # dados futuros não utilizados
    'readmissions_365_days_post_discharge',
    'readmissions_7_days_post_discharge',
    'last_albumin',
    'last_c_reactive_protein',
    'last_sodium',
    'last_urea',
    'last_hemoglobin',
    'last_creatinine',
    'last_white_blood_cell_count',
    'primary_diagnosis_at_discharge',
    'secondary_diagnosis_at_discharge'
]

df = df.drop(columns=colunas_retiradas)

### Correlação de Phik

In [ ]:
data_types = df.dtypes.to_dict()

data_types = {
    col: (
        'date' if pd.api.types.is_datetime64_any_dtype(df[col]) else
        'interval' if pd.api.types.is_numeric_dtype(df[col]) and df[col].nunique() >= 7 else
        'ordinal' if pd.api.types.is_numeric_dtype(df[col]) else
        'categorical'
    )
    for col in df.columns
}

# interval type variables need to be binned in order to calculate phik and the significance, so a list of interval variables is created
interval_cols = [col for col, v in data_types.items() if v=="interval" and col in df.columns]

##### Bin das variáveis intervalares

In [ ]:
data_binned, binning_dict = bin_data(df, cols=interval_cols, retbins=True)

binning_dict = {
    col: [(float(start), float(end)) for start, end in bins]
    for col, bins in binning_dict.items()
}

##### Cálculo da matriz de correlação e representação em heatmap

In [ ]:
phik_corr = data_binned.phik_matrix()

In [ ]:
correlacoes_tipo_alta = phik_corr['discharge_type']
correlacoes_tipo_alta = correlacoes_tipo_alta.drop(index=['discharge_type', 'length_of_stay', 'readmissions_30_days_post_discharge'])
correlacoes_tipo_alta = correlacoes_tipo_alta.sort_values(ascending=False)

In [ ]:
heatmap_data = pd.DataFrame(correlacoes_tipo_alta).T 

plt.figure(figsize=(19, 2))
sns.heatmap(heatmap_data, annot=True, cmap='inferno', vmin=0, vmax=1, cbar=True, square=True)
plt.title("Phik's Correlation - Death")
plt.savefig("correlacao_phik_death.png", bbox_inches='tight', dpi=300)


plt.show()

In [ ]:
correlacoes_reint = phik_corr['readmissions_30_days_post_discharge']
correlacoes_reint = correlacoes_reint.drop(index=['discharge_type', 'length_of_stay', 'readmissions_30_days_post_discharge'])
correlacoes_reint = correlacoes_reint.sort_values(ascending=False)

In [ ]:
heatmap_data = pd.DataFrame(correlacoes_reint).T 

plt.figure(figsize=(19, 2))
sns.heatmap(heatmap_data, annot=True, cmap='inferno', vmin=0, vmax=1, cbar=True, square=True)
plt.title("Phik's Correlation - Readmission")

plt.savefig("correlacao_phik_readmission.png", bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
correlacoes_tempoper = phik_corr['length_of_stay']
correlacoes_tempoper = correlacoes_tempoper.drop(index=['discharge_type', 'length_of_stay', 'readmissions_30_days_post_discharge'])
correlacoes_tempoper = correlacoes_tempoper.sort_values(ascending=False)

In [ ]:
heatmap_data = pd.DataFrame(correlacoes_tempoper).T 

plt.figure(figsize=(19, 2))
sns.heatmap(heatmap_data, annot=True, cmap='inferno', vmin=0, vmax=1, cbar=True, square=True)
plt.title("Phik's Correlation - Length of Stay")

plt.savefig("correlacao_phik_length.png", bbox_inches='tight', dpi=300)

plt.show()

##### Heatmap entre todas as variáveis

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(phik_corr, 
            annot=False,        
            cmap="coolwarm", 
            square=True,
            linecolor='black',    
            linewidths=0.5)

plt.title("Phik's Correlation Matrix between all variables")
plt.tight_layout()

plt.savefig("phiks_all.png", bbox_inches='tight', dpi=300)

plt.show()